The script below joins the address and property titles datasets (keeping only relevant columns)
<br>
<br>
The final output keeps the ids for parcels, titles, and addresses if further joins are required
<br>
<br>
Datasets used are provided by LINZ Data Service and licensed for reuse under the CC BY 4.0 licence
<br>
https://creativecommons.org/licenses/by/4.0/
<br>
https://data.linz.govt.nz/

In [ ]:
# Import pandas library
import pandas as pd

# Import datasets
# Note: Check filenames are same
title = pd.read_csv('nz-property-titles.csv')
address = pd.read_csv('nz-street-address.csv')
par_assoc = pd.read_csv('nz-title-parcel-association-list.csv')
add_ref = pd.read_csv('aims-address.csv')

In [ ]:
# Create clean address column
# E.g., '20 Main Street' instead of '1/20A Main Street'
address['clean_address'] =  address['address_number'].astype(str) + ' ' + address['full_road_name'] + ', ' +  address['suburb_locality'] + ', ' + address['town_city']
address['clean_address_2'] =  address['address_number'].astype(str) + ' ' + address['full_road_name'] + ', ' + address['suburb_locality']
address['clean_address'].fillna(address['clean_address_2'], inplace=True)

# Join tables
# Note: Sometimes different datasets have different columns names so check and change column names below as needed
address_2 = address[['address_id', 'full_address', 'clean_address']]
join_add_ref = pd.merge(address_2, add_ref[['address_id','parcel_id']], how='inner', on='address_id')
join_par_assoc = pd.merge(join_add_ref, par_assoc[['par_id', 'title_no']], how='inner', left_on='parcel_id', right_on='par_id')
join_title = pd.merge(join_par_assoc, title[['id', 'title_no']], how='right', on='title_no')

# Drop redundant column(s)
join_title = join_title.drop(columns=['parcel_id'])

# Mark duplicates
join_title['duplicate'] = join_title.duplicated(subset=['title_no'], keep=False)

# Remove duplicates
# Note: Only keeps one of the duplicates, E.g., 1/20A may be kept but 1/20B, 2/20C etc. will be dropped
drop_dups = join_title.drop_duplicates(subset=['title_no'], keep='first')

In [ ]:
## Join again with original datasets for extra columns
add_title = pd.merge(drop_dups, title[['id', 'type', 'land_district', 'issue_date', 'guarantee_status', 'estate_description']], how='left', on='id')

# If you want the property boundaries polygon column, comment the above and uncomment below
# add_title = pd.merge(df, title[['WKT', 'id', 'type', 'land_district', 'issue_date', 'guarantee_status', 'estate_description']], how='left', on='id')

add_address = pd.merge(address[['address_id', 'shape_X', 'shape_Y', 'full_road_name', 'suburb_locality', 'town_city']], add_title,  how='right', on='address_id')

In [ ]:
# Export to csv
add_address.to_csv('df.csv', index=False)